<a href="https://colab.research.google.com/github/LorenaFMarani/ATV1-DataMining/blob/main/Implementing_Spectral_Clustering_Using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando as bibliotecas

In [26]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.cluster import SpectralClustering , KMeans
from sklearn.preprocessing import StandardScaler, normalize 
from sklearn.decomposition import PCA 
from sklearn.metrics import silhouette_score 

Importando e limpando os dados (a coluna de ID, que é o código do cliente, é irrelevante neste caso).

In [27]:
credit_card = pd.read_csv('CC GENERAL.csv')
credit_card.drop(["CUST_ID"], axis=1, inplace=True)
credit_card = credit_card.fillna(method='ffill')
credit_card

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,1666.670542,0.636364,1499.00,1499.00,0.00,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,627.284787,0.000000,12
4,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8945,28.493517,1.000000,291.12,0.00,291.12,0.000000,1.000000,0.000000,0.833333,0.000000,0,6,1000.0,325.594462,48.886365,0.500000,6
8946,19.183215,1.000000,300.00,0.00,300.00,0.000000,1.000000,0.000000,0.833333,0.000000,0,6,1000.0,275.861322,48.886365,0.000000,6
8947,23.398673,0.833333,144.40,0.00,144.40,0.000000,0.833333,0.000000,0.666667,0.000000,0,5,1000.0,81.270775,82.418369,0.250000,6
8948,13.457564,0.833333,0.00,0.00,0.00,36.558778,0.000000,0.000000,0.000000,0.166667,2,0,500.0,52.549959,55.755628,0.250000,6


In [29]:
normalized_data = normalize(credit_card)
normalized_data

array([[3.93529208e-02, 7.87219116e-04, 9.17897286e-02, ...,
        1.34230246e-01, 0.00000000e+00, 1.15458778e-02],
       [2.93875725e-01, 8.34231054e-05, 0.00000000e+00, ...,
        9.84037362e-02, 2.03922922e-05, 1.10118488e-03],
       [3.10797802e-01, 1.24560826e-04, 9.63066935e-02, ...,
        7.81351110e-02, 0.00000000e+00, 1.49472991e-03],
       ...,
       [2.27729209e-02, 8.11047126e-04, 1.40538302e-01, ...,
        8.02142497e-02, 2.43314235e-04, 5.83954164e-03],
       [2.65239400e-02, 1.64244246e-03, 0.00000000e+00, ...,
        1.09890537e-01, 4.92732934e-04, 1.18255904e-02],
       [1.86405379e-01, 3.33425336e-04, 5.46775599e-01, ...,
        4.41566401e-02, 0.00000000e+00, 3.00082652e-03]])

Tratando a dimensão da base de dados

Como temos 17 colunas (*features*), é recomendável reduzir esta dimensão de forma que as principais informações não sejam perdidas. Para tanto, neste exemplo utilizaremos o PCA -  Principal Component Analysis

In [41]:
pca = PCA()
pca.fit(normalized_data)
explained_var = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, explained_var.shape[0] + 1),
    y=explained_var,
    labels={"x": "# Componentes", "y": "Discrepância"}
)


Para melhor visualização e performance deste estudo, teremos 3 componentes(colunas):

In [43]:
pca_data = PCA(n_components=3)
credit_pca = pd.DataFrame(pca_data.fit_transform(normalized_data), columns=['Componente 1','Componente 2','Componente 3'])
credit_pca

,Componente 1,Componente 2,Componente 3
0,-0.315576,-0.044823,0.024746
1,0.317328,-0.156866,0.368285
2,-0.206978,-0.183004,-0.123505
3,-0.279409,-0.118059,-0.127551
4,0.140988,-0.090323,-0.093426
...,...,...,...
8945,-0.290478,0.165477,0.036824
8946,-0.312760,0.151931,0.021563
8947,-0.387197,-0.061227,-0.004984
8948,-0.331117,-0.180086,0.087395


In [47]:
inertia = []
for i in range(1,10):
    cluster = KMeans(n_clusters=i)
    cluster.fit(credit_pca)
    inertia.append(cluster.inertia_)
px.line(inertia, title="Gráfico para definição de clusters no Spectral Clustering",labels={"index":"Clusters","value":"Inertia"})

Observando o gráfico acima, notamos que, à partir do 4, teremos retornos decrescentes, então é este que escolheremos.

AGRUPAMENTO ESPECTRAL


Visualização em 4 clusters (cores) com 3 componentes principais (x,y e z).

In [49]:
cluster_spectral = SpectralClustering(n_clusters=4, affinity='nearest_neighbors')
cluster_spectral.fit(credit_pca)
credit_card['Labels'] = cluster_spectral.labels_
px.scatter_3d(x=credit_pca['Componente 1'],y=credit_pca['Componente 2'],z=credit_pca['Componente 3'], color=cluster_spectral.labels_,size_max=18)

Testando a visualização de acordo com o saldo do cartão e compras realizadas

In [50]:
px.scatter(credit_card,x='BALANCE', y='PURCHASES',color='Labels',title='Saldo vs Compras')

Testando a visualização de acordo com o limite do cartão e pagamentos efetuados

In [51]:
px.scatter(credit_card,x='CREDIT_LIMIT', y='PAYMENTS',color='Labels', title='Limite de crédito vs Pagamentos')